In [1]:
# Limpio todas las variables
%reset -f

#  Procesamiento de Lenguaje Natural (NLP) de AI
### Consigna del desafío 1

<b id="1">Punto-1</b>. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos: Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

<b id="2">Punto-2</b>. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

<b id="3">Punto-3</b>. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

---

# Creación del DataSet de texto

En este ejercicio estamos tomando el texto de 5 archivos cortos .txt que contienen distintos tipos de información. el objetivo es crear un solo archivos con los requerimientos mínimos de un dataframe para que el modelo lo pueda entender. 

Contenido de archivos txt
- LOBO ESPACIAL WARHAMMER 40000 (novela)
- Deepfake: Todo lo que necesitas saber al respecto (articulo de noticia)
- Don Quijote (novela) 
- ¿Qué es la inteligencia artificial (IA)? (articulo de noticia) 
- Neuromante - William Gibson (novela) 

Los pasos para la preparación del Dataset 
1. Buscar información relevante como cantidad de palabras, párrafo, promedio. 
2. Generar Fragmentos del texto 
3. Asignación de etiquetas/clases

Nota: para este caso 

In [2]:
import os

def contar_palabras_y_parrafos(ruta_archivo):
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        texto = f.read()
    
    palabras = texto.split()
    total_palabras = len(palabras)
    
    parrafos = [p for p in texto.split('\n') if p.strip()]
    total_parrafos = len(parrafos)
    
    promedio_palabras_por_parrafo = total_palabras / total_parrafos if total_parrafos > 0 else 0
    
    return {
        "palabras": total_palabras,
        "parrafos": total_parrafos,
        "promedio_palabras_por_parrafo": promedio_palabras_por_parrafo
        
    }

def analizar_carpeta(ruta_carpeta):
    resultados = {}
    
    for archivo in os.listdir(ruta_carpeta):
        if archivo.endswith('.txt'):
            ruta_completa = os.path.join(ruta_carpeta, archivo)
            stats = contar_palabras_y_parrafos(ruta_completa)
            resultados[archivo] = stats
    
    return resultados

# Ruta a tu carpeta con los archivos .txt
ruta_carpeta = './DataSet/Lote-txt/'

# Analizar carpeta
resultados = analizar_carpeta(ruta_carpeta)

# Calcular promedios
total_palabras = sum(stats['palabras'] for stats in resultados.values())
total_parrafos = sum(stats['parrafos'] for stats in resultados.values())
archivos_count = len(resultados)
promedio_palabras = total_palabras / archivos_count
promedio_parrafos = total_parrafos / archivos_count

# Mostrar resultados
print(" Estadísticas de los archivos de texto:\n")
for archivo, stats in resultados.items():
    porcentaje_palabras = (stats['palabras'] / total_palabras) * 100
    print(f"Archivo: {archivo}")
    print(f"  Palabras: {stats['palabras']}  ({porcentaje_palabras:.1f}% del total )")
    print(f"  Párrafos: {stats['parrafos']}")
    print(f"  promedio_palabras_por_parrafo: {stats['promedio_palabras_por_parrafo']:.2f}")
    print("-" * 40)


print("\n*** Promedio TOTAL ***")
print(f"  Promedio de palabras entre todos los archivo: {promedio_palabras:.0f}")
print(f"  Promedio de párrafos entre todos los archivo: {promedio_parrafos:.0f}")

 Estadísticas de los archivos de texto:

Archivo: texto-1.txt
  Palabras: 103027  (43.6% del total )
  Párrafos: 1792
  promedio_palabras_por_parrafo: 57.49
----------------------------------------
Archivo: texto-2.txt
  Palabras: 5710  (2.4% del total )
  Párrafos: 165
  promedio_palabras_por_parrafo: 34.61
----------------------------------------
Archivo: texto-3.txt
  Palabras: 29835  (12.6% del total )
  Párrafos: 798
  promedio_palabras_por_parrafo: 37.39
----------------------------------------
Archivo: texto-4.txt
  Palabras: 4480  (1.9% del total )
  Párrafos: 89
  promedio_palabras_por_parrafo: 50.34
----------------------------------------
Archivo: texto-5.txt
  Palabras: 93365  (39.5% del total )
  Párrafos: 3129
  promedio_palabras_por_parrafo: 29.84
----------------------------------------

*** Promedio TOTAL ***
  Promedio de palabras entre todos los archivo: 47283
  Promedio de párrafos entre todos los archivo: 1195


### Creación del DataSet

El siguiente código esta personalizado para tomar 5 textos en archivo plano .txt que realizaran la creación del dataset utilizando la librería de panda en la generación de las etiquetas y el texto. 


| Sintaxis numpy| Sintaxis panda|
|---|---|
| `df_train.data` | `df_train['texto']` |
| `df_train.target` | `df_train['etiqueta']` |
| `newsgroups_train = fetch_20newsgroups(...)` | `df_train = df` |
| `newsgroups_test = fetch_20newsgroups(...)` | `df_test = df` |







En la creación del dataset tenemos que tomar en cuneta 2 valores muy importantes tam_fragmento y overlap pero antes repasemos las siguientes definiciones. 

- ETIQUETA/CLASE (Label/Class): Categoría o tipo al que pertenece un document, podemos utiliza la analogía de El género/tema del libro,  en este proyecto es :
        - texto-1.txt: lobo_espacial
        - texto-2.txt: deepfake
        - texto-3.txt: don_quijote
        - texto-4.txt: ia_articulo
        - texto-5.txt: neuromante'
- CORPUS: Conjunto completo de todos los Document / fragmentos de todos los archivos
- Document : Unidad básica con un bloque de texto que se analiza independientemente, su analogía un párrafo o capítulo del libro. Técnicamente Una fila en un DataFrame, un vector en la matriz documento-término. Para definir un Document tenemos que indicar 
    - tam_fragmento (Fragment Size): Número de palabras que contiene cada documento/fragmento. ejemplo `fragmentos = fragmentar_texto(texto_limpio, tam_fragmento=300,` indicamos un bloque de 300 palabras. 
    - OVERLAP (Solapamiento): Número de palabras compartidas entre fragmentos consecutivos Ejemplo :


|Sin Overlap (overlap=0):|       |                |                    |       |
|----------------|--------       |  ---------     |   ---------         | ---- |
|Texto original: | [1][2][3][4]  |[5][6][7][8][9] |[10][11][12]...      |
|Fragmento 1:    |[1][2][3][4][5]|                |                     | (palabras 1-5)| 
|Fragmento 2:    |               |[6][7][8][9][10]|                     |  (palabras 6-10)|
|Fragmento 3:    |               |                |[11][12][13][14][15] | (palabras 11-15) |

.


|Con Overlap=2 palabras:|        |          |           |       |
|----------------|-------------- |-------- | ----------|  ---     |
|Texto original: |[1][2][3][4][5]|[6][7][8]|[9][10][11]|[12]...|
|Fragmento 1:    |[1][2][3] [4][5]||                     |  (palabras 1-5)|         
|Fragmento 2:    |         [4][5]|[6][7][8]|           |  (palabras 4-8)← Repite 4,5|
|Fragmento 3:    |                   [7][8]|[9][10][11]|  (palabras 7-11) ← Repite 7,8|




In [3]:
# +++++++++++++++ Creación del Dataframe +++++++++++++++++++++
import pandas as pd
import os
import re
from collections import Counter

def leer_archivos_txt(directorio):
    """
    Lee todos los archivos .txt del directorio especificado
    """
    archivos_datos = {}
    
    # Mapeo de nombres de archivo a etiquetas / Clases
    mapeo_etiquetas = {
        'texto-1.txt': 'lobo_espacial',
        'texto-2.txt': 'deepfake', 
        'texto-3.txt': 'don_quijote',
        'texto-4.txt': 'ia_articulo',
        'texto-5.txt': 'neuromante'
    }
    
    for archivo in os.listdir(directorio):
        if archivo.endswith('.txt'):
            ruta_completa = os.path.join(directorio, archivo)
            try:
                with open(ruta_completa, 'r', encoding='utf-8') as f:
                    contenido = f.read()
                    etiqueta = mapeo_etiquetas.get(archivo, 'desconocido')
                    archivos_datos[archivo] = {
                        'contenido': contenido,
                        'etiqueta': etiqueta
                    }
                print(f" Archivo leído: {archivo} -> Etiqueta: {etiqueta}")
            except Exception as e:
                print(f"RERRO:  Error leyendo {archivo}: {e}")
    
    return archivos_datos

def limpiar_texto(texto):
    """
    Limpieza básica del texto
    """
    # Normalizar espacios en blanco
    texto = re.sub(r'\s+', ' ', texto)
    # Eliminar espacios al inicio y final
    texto = texto.strip()
    return texto

def contar_palabras(texto):
    """
    Cuenta las palabras en un texto
    """
    return len(texto.split())

#def fragmentar_texto(texto, tam_fragmento=300, overlap=150):
def fragmentar_texto(texto, tam_fragmento=600, overlap=300):
    """
    Fragmenta el texto en chunks con overlap
    
    Args:
        texto: texto a fragmentar
        tam_fragmento: número de palabras por fragmento
        overlap: número de palabras de solapamiento
    
    Returns:
        lista de fragmentos
    """
    palabras = texto.split()
    fragmentos = []
    
    avance = tam_fragmento - overlap  # palabras nuevas en cada paso
    
    for i in range(0, len(palabras), avance):
        fragmento_palabras = palabras[i:i + tam_fragmento]
        
        # Solo agregar si tiene al menos un mínimo de palabras
        if len(fragmento_palabras) >= 50:  # mínimo 50 palabras por fragmento
            fragmento_texto = ' '.join(fragmento_palabras)
            fragmentos.append({
                'texto': fragmento_texto,
                'num_palabras': len(fragmento_palabras),
                'posicion_inicio': i,
                'posicion_fin': i + len(fragmento_palabras)
            })
        
        # Si el último fragmento no llega al tamaño mínimo, parar
        if i + tam_fragmento >= len(palabras):
            break
    
    return fragmentos

def crear_dataset(directorio_input="./DataSet/Lote-txt", archivo_output="./DataSet/dataset_nlp.csv"):
    """
    Función principal para crear el dataset
    """
    print(" Iniciando creación del dataset...")
    print("=" * 50)
    
    # 1. Carga de archivos
    print("Paso 1: Cargando archivos...")
    archivos_datos = leer_archivos_txt(directorio_input)
    
    if not archivos_datos:
        print("ERRO: No se encontraron archivos .txt en el directorio")
        return None
    
    print(f" Se cargaron {len(archivos_datos)} archivos")
    print()
    
    # 2. Fragmentación
    print("Paso 2: Fragmentando textos...")
    dataset_rows = []
    
    for nombre_archivo, datos in archivos_datos.items():
        print(f"Procesando: {nombre_archivo}")
        
        texto_limpio = limpiar_texto(datos['contenido'])
        palabras_totales = contar_palabras(texto_limpio)
        
        #fragmentos = fragmentar_texto(texto_limpio, tam_fragmento=300, overlap=150)
        fragmentos = fragmentar_texto(texto_limpio, tam_fragmento=600, overlap=300)
        
        print(f"   Texto original: {palabras_totales} palabras")
        print(f"   Fragmentos generados: {len(fragmentos)}")
        
        # Agregar cada fragmento al dataset
        for idx, fragmento in enumerate(fragmentos):
            dataset_rows.append({
                'texto': fragmento['texto'],
                'etiqueta': datos['etiqueta'],
                'archivo_origen': nombre_archivo,
                'fragmento_id': idx + 1,
                'num_palabras': fragmento['num_palabras'],
                'posicion_inicio': fragmento['posicion_inicio'],
                'posicion_fin': fragmento['posicion_fin']
            })

    
    # 3. Crear DataFrame
    print(f"\n  Paso 3: Creando DataFrame...")
    # texto,etiqueta,archivo_origen,fragmento_id,num_palabras,posicion_inicio,posicion_fin
    df = pd.DataFrame(dataset_rows)
    # nota de conceptos 
    # --- Cada fragmento es tratado como un "documento" independiente
    # --- Cada fragmento tiene su propia etiqueta y características
    print(f'       Dataset creado con {len(df)} fragmentos - {len(df)} "documentos" para el modelo')
    print(f"Estrutura -> texto,etiqueta,archivo_origen,fragmento_id,num_palabras,posicion_inicio,posicion_fin")
    
    # 4. Guardar
    print("  Paso 4: Guardando dataset...")
    
    # Crear directorio si no existe
    directorio_output = os.path.dirname(archivo_output)
    if directorio_output and not os.path.exists(directorio_output):
        os.makedirs(directorio_output)
    
    df.to_csv(archivo_output, index=False, encoding='utf-8')
    print(f"   Dataset guardado en: {archivo_output}")
    
    return df



# ++++++++++++++++++ Descomentar si no se tiene el dataset para este ejercicio
# llamada a la función principal para la creación del dataset 
#dataset = crear_dataset()






### Carga del dataset

In [4]:
""" 
Es un DataFrame de pandas 
Tiene columnas con nombres:
-  'texto' (equivale a .data)
-  'etiqueta' (equivale a .target pero con strings)
-  No tiene equivalente directo a .target_names
- Estructura de "tabla con columnas" .. NO  Estructura de "objeto con atributos" dataset de sklearn
"""
import pandas as pd

#cargamos del datafame  panda
#archivo_csv="./DataSet/dataset_nlp300.csv" # tam_fragmento=300, overlap=150)
archivo_csv="./DataSet/dataset_nlp.csv" # tam_fragmento=600, overlap=300)
df = pd.read_csv(archivo_csv, encoding='utf-8')

In [5]:
# ---------------- Info Complementaria ------------------------

# Funcipon de estadisticas 
def mostrar_estadisticas(df):
    """
    Muestra estadísticas descriptivas del dataset
    """   
    # Distribución por clases
    print(" Distribución por clases:")
    distribucion = df['etiqueta'].value_counts()
    for etiqueta, cantidad in distribucion.items():
        porcentaje = (cantidad / len(df)) * 100
        print(f"  {etiqueta}: {cantidad} fragmentos ({porcentaje:.1f}%)")
    
    print()
    
    # Estadísticas de longitud
    print(" Estadísticas de longitud (palabras):")
    print(f" - Promedio: {df['num_palabras'].mean():.1f} palabras")
    print(f" - Mediana: {df['num_palabras'].median():.1f} palabras")
    print(f" - Mínimo: {df['num_palabras'].min()} palabras")
    print(f" - Máximo: {df['num_palabras'].max()} palabras")
    
    print()
    
    # Distribución por archivo origen
    print("Fragmentos por archivo origen:")
    por_archivo = df['archivo_origen'].value_counts()
    for archivo, cantidad in por_archivo.items():
        print(f"  {archivo}: {cantidad} fragmentos(documentos)")
    
    print()
    
    # Muestra del dataset
    print("Muestra del dataset (primeros 3 fragmentos):")
    print(df[['etiqueta', 'archivo_origen', 'fragmento_id', 'num_palabras']].head(3).to_string(index=False))
    print()

#  Estadísticas
print(f"Info  del dataset... \n")
mostrar_estadisticas(df)


Info  del dataset... 

 Distribución por clases:
  lobo_espacial: 343 fragmentos (43.6%)
  neuromante: 311 fragmentos (39.6%)
  don_quijote: 99 fragmentos (12.6%)
  deepfake: 19 fragmentos (2.4%)
  ia_articulo: 14 fragmentos (1.8%)

 Estadísticas de longitud (palabras):
 - Promedio: 598.9 palabras
 - Mediana: 600.0 palabras
 - Mínimo: 310 palabras
 - Máximo: 600 palabras

Fragmentos por archivo origen:
  texto-1.txt: 343 fragmentos(documentos)
  texto-5.txt: 311 fragmentos(documentos)
  texto-3.txt: 99 fragmentos(documentos)
  texto-2.txt: 19 fragmentos(documentos)
  texto-4.txt: 14 fragmentos(documentos)

Muestra del dataset (primeros 3 fragmentos):
     etiqueta archivo_origen  fragmento_id  num_palabras
lobo_espacial    texto-1.txt             1           600
lobo_espacial    texto-1.txt             2           600
lobo_espacial    texto-1.txt             3           600



In [6]:
# fragmento de texto. 
Texto = df['texto'].iloc[2]
print(f" Trexto fragmento 3 : {Texto} \n")

## etiqueta numero 5 
#etiqueta_doc = df['etiqueta'].iloc[5]
#print(f' etiqueta / clases : {etiqueta_doc} \n')

## Ver todas las clases/ etiquetas  
clases = df['etiqueta'].unique()
print(f' etiqueta / clases : {clases} \n')

# cuántos documentos hay por clase
distribucion_clases = df['etiqueta'].value_counts()
print(f'documentos hay por clase:  {distribucion_clases}  \n')

# 5. Ver información completa del fragmento 
info_doc5 = df.iloc[2]
print(f' {info_doc5} \n')

## 6. Ver todos los documentos de una clase específica
#docs_don_quijote = df[df['etiqueta'] == 'don_quijote']
#print(f"\n Documentos de Don Quijote: {len(docs_don_quijote)} \n")

# 7. Ver primer texto de cada clase
print (f'\n primer texto de cada clase :')
for clase in df['etiqueta'].unique():
    primer_texto = df[df['etiqueta'] == clase]['texto'].iloc[0]
    print(f"Clase {clase}: {primer_texto[:100]}...")

 Trexto fragmento 3 : de ser tan potentes como aquéllos, pero pese a todas las cosas no parecían estar tan mal. A decir verdad estaba disfrutando con aquella situación. Después de un mes de meditación en su celda de El Colmillo y de una semana encerrado a bordo de uno de los grandes cruceros estelares del Imperio en ruta hacia esta guerra menor, estaba ansioso por entrar en acción. En realidad, no era nada sorprendente porque había nacido para ello, y había sido entrenado con esa finalidad. Toda su vida había sido una preparación para este momento. Después de todo, era un Marine Espacial Imperial del Capítulo de los Lobos Espaciales. ¿Qué otra cosa podría pedirle a la vida sino esto? Tenía un bólter cargado en sus manos y, ante sí, a los enemigos del Emperador. En esta vida no había mayor placer que cumplir con el deber y acabar con la vida de aquellos lamentables herejes. La pared de ladrillos que tenía a su espalda se estremecía. Trozos de piedra golpearon su armadura, lo que le hizo

<h3><a href="#1">Punto 1</a></h3>

- Vectorizar documentos. 
- medir similaridad con el resto de los documentos
    - Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.


In [7]:
# utilizamos el 100% de los datos del DataFrame para entrenamiento y testing  
df_train = df
df_test = df


In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(df_train['texto']) 

In [9]:

print(type(X_train))
print(f'shape: {X_train.shape} documentos, términos')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]} ')



<class 'scipy.sparse._csr.csr_matrix'>
shape: (786, 20713) documentos, términos
Cantidad de documentos: 786
Tamaño del vocabulario (dimensionalidad de los vectores): 20713 


In [10]:
print(X_train)

  (0, 20526)	0.10283491815316567
  (0, 11575)	0.11838698006512331
  (0, 12083)	0.023018584485384623
  (0, 8201)	0.0304721218691385
  (0, 20517)	0.059193490032561656
  (0, 86)	0.059193490032561656
  (0, 12084)	0.02495721654982468
  (0, 8202)	0.029781788344500487
  (0, 7932)	0.10662229938989885
  (0, 16197)	0.10662229938989885
  (0, 7933)	0.11838698006512331
  (0, 49)	0.042094170224988015
  (0, 11)	0.046429252666984264
  (0, 20)	0.0455351187389706
  (0, 19777)	0.044726276214652534
  (0, 14109)	0.04103198282059592
  (0, 18477)	0.059193490032561656
  (0, 20531)	0.059193490032561656
  (0, 48)	0.059193490032561656
  (0, 19416)	0.051417459076582836
  (0, 7262)	0.059193490032561656
  (0, 13375)	0.059193490032561656
  (0, 52)	0.059193490032561656
  (0, 7093)	0.046429252666984264
  (0, 6591)	0.0455351187389706
  :	:
  (785, 17282)	0.04350315192820584
  (785, 16677)	0.08020703154216098
  (785, 9111)	0.07103106163867219
  (785, 18951)	0.07242583744936117
  (785, 19081)	0.0640065378459706
  (785, 9

In [11]:
print(f" La palabra espacio esta en la posición --   {tfidfvect.vocabulary_['espacio']} -- del vector ")

 La palabra espacio esta en la posición --   8203 -- del vector 


In [12]:
# crea un diccionario inverso que mapea índices a palabras de manera Inversa  índices de las palabras de vuelta a texto
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()} 
algunas_palabras = dict(list(idx2word.items())[:10])
print(f'ALgunas palabras y su indice : {algunas_palabras}')

ALgunas palabras y su indice : {20526: 'william', 11575: 'king', 12083: 'lobo', 8201: 'espacial', 20517: 'warhammer', 86: '40000', 12084: 'lobos', 8202: 'espaciales', 7932: 'epub', 16197: 'r1'}


In [13]:
print(f"Tamaño del vocabulario: {len(tfidfvect.vocabulary_)} palabras")

Tamaño del vocabulario: 20713 palabras


In [14]:
print(f"Indicas 10 de algunas palabras : {list(tfidfvect.vocabulary_.keys())[:10]}")

Indicas 10 de algunas palabras : ['william', 'king', 'lobo', 'espacial', 'warhammer', '40000', 'lobos', 'espaciales', 'epub', 'r1']


In [15]:
def analizar_indices_por_clase(df):
    """
    indica el rango de los indice `idx` de los documentos (fragmentos) de cada etiqueta
    """
    
    clases_ordenadas = df['etiqueta'].unique()  # Orden de aparición en el dataset
    
    for clase in clases_ordenadas:
        # Obtener todos los índices de esa clase
        indices_clase = df[df['etiqueta'] == clase].index.tolist()
        
        min_idx = min(indices_clase)
        max_idx = max(indices_clase)
        cantidad = len(indices_clase)
        
        print(f" {clase}:")
        print(f"   Rango: del idx de los fragmentos {min_idx} al {max_idx}")
        print(f"   Cantidad: {cantidad} documentos")

analizar_indices_por_clase(df)

 lobo_espacial:
   Rango: del idx de los fragmentos 0 al 342
   Cantidad: 343 documentos
 deepfake:
   Rango: del idx de los fragmentos 343 al 361
   Cantidad: 19 documentos
 don_quijote:
   Rango: del idx de los fragmentos 362 al 460
   Cantidad: 99 documentos
 ia_articulo:
   Rango: del idx de los fragmentos 461 al 474
   Cantidad: 14 documentos
 neuromante:
   Rango: del idx de los fragmentos 475 al 785
   Cantidad: 311 documentos


### Similitud Coseno `cosine_similarity`


**La Similitud Coseno** es una métrica fundamental en el Procesamiento del Lenguaje Natural (PLN) que se utiliza para medir cuán similares son dos documentos de texto. No mide la magnitud de los vectores, sino la orientación de sus vectores. Imagina cada documento como un vector en un espacio multidimensional; la similitud coseno _calcula el coseno del ángulo_ entre estos dos vectores. 

La idea central es que, si dos documentos que tratan sobre temas similares, es probable que utilicen un vocabulario similar, lo que resultaría en vectores que apuntan en direcciones parecidas en el espacio vectorial. 
 

En en siguiente ejercicio estamos realizando la Similitud de Coseno en el fragmento 10 `idx= 10` contra todos los fragmentos (documentos) de los 5 artículos que contienen distinta información entre los mismos, pero dentro de los fragmentos correspondiente a dichos artículos observamos un valor en la la similaridad.

In [16]:
# definimos Fragmento 
idx= 10
# vemos a que Etiqueta corresponde 
print (f' Etiquetas / Clases:   {df_train['etiqueta'].iloc[idx]} \n')
# Vemos el Texto almacenado en el fragmento 
df_train['texto'].iloc[idx]

 Etiquetas / Clases:   lobo_espacial 



'en aquella extraña sensación. Sabía lo que podía esperar o debía saberlo: al fin y al cabo había muerto anteriormente. Una helada claridad se apoderó de su espíritu, y su memoria se remontó en el pasado mientras su alma se aventuraba siglos atrás, recordando. UNO EL ÚLTIMO BALUARTE —¡Vamos a morir todos! —gritó Yorvik el Arponero, alumbrando en derredor y con los ojos muy abiertos por el miedo. Un relámpago cruzó el cielo de Fenris e iluminó la cara atormentada del hombre. El profundo terror hizo que su alarido se oyese, incluso, por encima del rugido del viento y del ruido atronador de las olas que chocaban contra el barco. Las gotas de la lluvia que azotaba su rostro resbalaban como lágrimas. —¡Quédate callado! —contestó Ragnar mientras abofeteaba la cara del aterrorizado hombre. Estupefacto por haber sido golpeado por un jovencito que apenas tenía edad para lucir barba en las mejillas, Yorvik echó mano de su hacha, olvidado momentáneamente de su miedo. Ragnar movió la cabeza y clav

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print (f' Calcular Similaridad coseno entre el fragmento --{idx}-- que pertenece a la Etiquetas/Clases: --{df_train['etiqueta'].iloc[idx]}-- con todos los documentos (fragmentos )\n')
## Calcular Similaridad coseno entre el fragmento idx con todos los documentos (fragmentos )
cossim = cosine_similarity(X_train[idx], X_train)[0]
# Ordena los valores de similaridad de mayor a menor
np.sort(cossim)[::-1] # todos documentos 

# Vemos  cuáles son los valores de similaridad más altos NO indica qué documentos tienen esos vectores



 Calcular Similaridad coseno entre el fragmento --10-- que pertenece a la Etiquetas/Clases: --lobo_espacial-- con todos los documentos (fragmentos )



array([1.        , 0.72787623, 0.71415594, 0.47912957, 0.4603731 ,
       0.4563406 , 0.45330197, 0.44571104, 0.43744339, 0.43196383,
       0.42254724, 0.41899589, 0.41705214, 0.41675146, 0.41377503,
       0.41236456, 0.41220132, 0.41211592, 0.41130716, 0.40811557,
       0.40734294, 0.40633294, 0.40591669, 0.40475013, 0.40405163,
       0.40351019, 0.40271437, 0.40253351, 0.40248306, 0.40150139,
       0.39982681, 0.39962146, 0.39958535, 0.39943688, 0.3988447 ,
       0.39854427, 0.39649409, 0.39630514, 0.39624484, 0.39611708,
       0.39592964, 0.39583402, 0.3946352 , 0.39424896, 0.39423572,
       0.39414219, 0.3935196 , 0.39316394, 0.39251415, 0.39226937,
       0.39212065, 0.39158939, 0.38956175, 0.38924259, 0.38888524,
       0.38874781, 0.38834659, 0.38810721, 0.38808609, 0.38806783,
       0.38784481, 0.38780766, 0.38769816, 0.38758601, 0.38738744,
       0.38705703, 0.38697786, 0.38644617, 0.3863802 , 0.38618373,
       0.38582311, 0.38555636, 0.38536979, 0.38472787, 0.38418

como podemos observar en los resultados obtenidos en la _Similaridad entre el fragmento 10 contra todos los fragmentos_.
- EL valor 1.0: como la **similaridad  perfecta** asumimos que estamos analizando el mismo fragmento 10 contra si mismo
- Podemos asumir que los rangos entre 0.7 y 0.4 son los documentos que pertenecen al documento _lobo_espacial_
- El resto de valores enter 0.3 y 0.1 que tienen un valor de similaridad muy baja podemos asumir que pertenecen al resto de los documentos. 

En los siguientes códigos mostramos otros códigos para mostrar los valores asignados a los fragmentos, similitud e identificación de los mismos. 

In [18]:
# Ordena los índice según los valores de similaridad de mayor a menor
# Te dice qué documentos tienen las similaridades más altas
np.argsort(cossim)[::-1]

array([ 10,   9,  11,  14,  15,  13,  16,  12,  17,  76,  69,  38,  35,
        46,  45,  22, 243,  54,  89, 317,  88,  52, 158, 105,  53,  70,
        79,  57, 244, 280,  66, 486, 159,  60,  68,  21,  75,  39,  41,
       221, 237, 236,  64, 337, 230, 268,  34,  58, 124, 213,  67,  86,
       272, 162,  37, 153,  72,  49, 316, 127,  80,  29, 269, 233,  73,
       315,  65, 209, 290,  77, 163, 239, 180,  87,  51, 176,   7,   8,
       185, 304, 175, 131, 487,  63, 208,  25,  18, 314, 302,  61, 242,
       279, 657, 128,  59, 291, 303, 186, 311, 324, 222, 245, 501,  36,
        24, 281, 179, 748, 223, 238, 774,  92, 220,  26,  48, 267, 313,
       318, 273, 106,  32, 295,  85, 140, 301,  44, 294,  40, 338, 500,
       207, 541, 161, 130, 169, 312, 232, 143, 725, 104,  28, 134, 212,
       157, 231, 139,  91, 747,  47, 559, 166, 658, 256, 202, 154, 775,
       227, 693, 210, 164, 310,  55, 206, 150, 326, 260, 203, 694, 192,
       178,  20,  23,  71,  84, 229, 339, 327,  43, 146,  27, 17

In [19]:
# Calculamos valor de similitud contra fragmentos específicos de multiples documentos
idx_doc_lista = [10, 9, 11,475,785, 362,343,460,461,474,361]  # Lista de documentos a comparar

for idx_doc in idx_doc_lista:
    # CALCULAR SIMILITUD DIRECTA ENTRE DOS DOCUMENTOS
    similitud_doidx = cosine_similarity(X_train[idx], X_train[idx_doc])[0][0]
    print(f'Comparamos fragmento "{idx}" con Etiquetas/Clases "{df_train['etiqueta'].iloc[idx]}" donde el valor de similaridad es de {similitud_doidx:.4f} contra el fragmento "{idx_doc}" de "{df_train['etiqueta'].iloc[idx_doc]}"')

Comparamos fragmento "10" con Etiquetas/Clases "lobo_espacial" donde el valor de similaridad es de 1.0000 contra el fragmento "10" de "lobo_espacial"
Comparamos fragmento "10" con Etiquetas/Clases "lobo_espacial" donde el valor de similaridad es de 0.7279 contra el fragmento "9" de "lobo_espacial"
Comparamos fragmento "10" con Etiquetas/Clases "lobo_espacial" donde el valor de similaridad es de 0.7142 contra el fragmento "11" de "lobo_espacial"
Comparamos fragmento "10" con Etiquetas/Clases "lobo_espacial" donde el valor de similaridad es de 0.3138 contra el fragmento "475" de "neuromante"
Comparamos fragmento "10" con Etiquetas/Clases "lobo_espacial" donde el valor de similaridad es de 0.3066 contra el fragmento "785" de "neuromante"
Comparamos fragmento "10" con Etiquetas/Clases "lobo_espacial" donde el valor de similaridad es de 0.2803 contra el fragmento "362" de "don_quijote"
Comparamos fragmento "10" con Etiquetas/Clases "lobo_espacial" donde el valor de similaridad es de 0.2237 

In [20]:
# ----------------#Toma los índices ordenados por similaridad y limitando la cantidad de resultado- 
#Excluye el primero [1:] (el documento consigo mismo)
#Toma los siguientes 5 [1:6]
#Resultado: [0, 3, 4, 1] (los 4 más similares, excluyendo él mismo)
#¿Por qué excluir el primero?
#El documento más similar a sí mismo es él mismo (similaridad = 1.0)
mostsim = np.argsort(cossim)[::-1][1:6] # son los documentos MÁS similares (excluyendo él mismo)
print (mostsim)

[ 9 11 14 15 13]


In [21]:
# Listado por rango de similitud top 10 

#idx = 120
# Etiqueta del documento de consulta
etiqueta_consulta = df_train['etiqueta'].iloc[idx]
print(f"Etiqueta del Documento de consulta: '{etiqueta_consulta}' (fragmento:  {idx})\n")

# Calcula la similitud entre un documento y todos los demás
cossim = cosine_similarity(X_train[idx], X_train)[0]
# Obtener índices ordenados por similitud (de mayor a menor)
# no excluimos el documento de similarity 1, estamos consiente que es un resultado obtenido por compararse a sí mismo es él mismo
indices_similitud = np.argsort(cossim)[::-1]
# Obtener valores de similitud ordenados
documentos_similitud = np.sort(cossim)[::-1]

# Mostrar los 10 documentos más similares
## print (f' Etiquetas / Clases:   {df_train['etiqueta'].iloc[10]} ')   # individual 
# grupal 
for i in range(10):  # Top 10
    doc_idx = indices_similitud[i]
    etiqueta = df_train['etiqueta'].iloc[doc_idx]
    valor_sim = documentos_similitud[i]
    print(f"Etiquetas / Clases: {etiqueta} -> documento: {doc_idx} -> valor de similaridad: {valor_sim:.4f}")

Etiqueta del Documento de consulta: 'lobo_espacial' (fragmento:  10)

Etiquetas / Clases: lobo_espacial -> documento: 10 -> valor de similaridad: 1.0000
Etiquetas / Clases: lobo_espacial -> documento: 9 -> valor de similaridad: 0.7279
Etiquetas / Clases: lobo_espacial -> documento: 11 -> valor de similaridad: 0.7142
Etiquetas / Clases: lobo_espacial -> documento: 14 -> valor de similaridad: 0.4791
Etiquetas / Clases: lobo_espacial -> documento: 15 -> valor de similaridad: 0.4604
Etiquetas / Clases: lobo_espacial -> documento: 13 -> valor de similaridad: 0.4563
Etiquetas / Clases: lobo_espacial -> documento: 16 -> valor de similaridad: 0.4533
Etiquetas / Clases: lobo_espacial -> documento: 12 -> valor de similaridad: 0.4457
Etiquetas / Clases: lobo_espacial -> documento: 17 -> valor de similaridad: 0.4374
Etiquetas / Clases: lobo_espacial -> documento: 76 -> valor de similaridad: 0.4320


<h3><a href="#2">Punto 2</a></h3>

### Modelo de clasificación Naïve Bayes

Naïve Bayes es un algoritmo probabilístico que clasifica documentos calculando qué tan probable es que pertenezcan a cada clase, basándose en las palabras que contienen.
- Modelo de Naïve Bayes Multinomial. Es la Versión "clásica" de Naïve Bayes para texto, es bueno cuando tiene las clases balanceadas, Cuenta frecuencias de palabras directamente Aprendiendo "¿Qué palabras aparecen en documentos de clase X?"

    Ejemplo el modelo para clasificar un documento, pregunta, ¿Qué tan probable es que 'armadura' aparezca en lobo_espacial?, ¿Qué tan probable es que 'ciberespacio' aparezca en neuromante?

- Modelo de Naïve Bayes ComplementNB. Versión mejorada diseñada para clases desbalanceadas, En lugar de aprender de documentos de la clase, aprende de documentos de todas las otras clases aprendiendo  "¿Qué palabras NO aparecen en las otras clases?"

    Ejemplo el modelo para clasificar como 'lobo_espacial', pregunta: ¿Qué tan diferentes son estas palabras de neuromante, don_quijote, etc.?

=================================0

- Multinomial -> Funciona bien con CountVectorizer
- ComplementNB ->  Funciona bien con TF-IDF

In [22]:
# librerías 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# ==========================================
# Train/Test (100% de los datos)
# ==========================================
## Parámetros del Vectorizador (TfidfVectorizer)
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(df_train['texto'])
X_test = tfidfvect.transform(df_test['texto'])  # Solo transform, no fit

# Etiquetas
y_train = df_train['etiqueta']
y_test = df_test['etiqueta']

print("Datos preparados para los modelos Naïve Bayes:")
print(f"   X_train shape: {X_train.shape}")
print(f"   X_test shape: {X_test.shape} \n")
print(f"   Clases: {list(df_train['etiqueta'].unique())}")
print()



Datos preparados para los modelos Naïve Bayes:
   X_train shape: (786, 20713)
   X_test shape: (786, 20713) 

   Clases: ['lobo_espacial', 'deepfake', 'don_quijote', 'ia_articulo', 'neuromante']



In [23]:
# ==========================================
# Modelo Multinomial Naive Bayes
# ==========================================

print("*** MULTINOMIAL NAIVE BAYES ***")

# Instanciar y entrenar modelo
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

# Predicciones
y_pred_mnb = mnb.predict(X_test)

# F1-score macro
f1_mnb = f1_score(y_test, y_pred_mnb, average='macro')

print(f"F1-score (macro): {f1_mnb}")
print(f"Predicciones realizadas: {len(y_pred_mnb)}")
print()



*** MULTINOMIAL NAIVE BAYES ***
F1-score (macro): 0.5577733084525537
Predicciones realizadas: 786



**¿Por qué vemos que  ComplementNB superó a MultinomialNB?**

ComplementNB **tiene un mejor Manejo de clases desbalanceadas:**
- ComplementNB fue diseñado específicamente para corregir problemas de MultinomialNB
- Es más robusto cuando hay clases con pocos ejemplos
- En nuestro ejemplo tenemos un dataset desbalanceado algunas clases tienen más fragmentos que otras

En comparación entre los 2 modelos tenemos que
- **MultinomialNB:**
  - Estima probabilidades usando SOLO ejemplos de cada clase
  - Puede sufrir de "sesgo" hacia clases dominantes
- **ComplementNB:**
  - Estima probabilidades usando ejemplos de TODAS las otras clases
  - Más equilibrado y menos propenso a sesgos


In [24]:

# ==========================================
# Modelo Complement Naive Bayes
# ==========================================

print("*** COMPLEMENT NAIVE BAYES***")
print("-" * 30)

# Instanciar y entrenar modelo
cnb = ComplementNB()
cnb.fit(X_train, y_train)

# Predicciones
y_pred_cnb = cnb.predict(X_test)

# F1-score macro
f1_cnb = f1_score(y_test, y_pred_cnb, average='macro')

print(f"F1-score (macro): {f1_cnb}")
print(f"Predicciones realizadas: {len(y_pred_cnb)}")


*** COMPLEMENT NAIVE BAYES***
------------------------------
F1-score (macro): 1.0
Predicciones realizadas: 786


#### Tipos de vectorización  en los modelos 
Algoritmo: Naïve Bayes (la lógica matemática)
Modelos: MultinomialNB y ComplementNB (implementaciones específicas)
Vectorizadores: CountVectorizer y TF-IDF (preprocesamiento de texto)



- CountVectorizer. Convierte texto en números contando cuántas veces aparece cada palabra.
- TF-IDF (Term Frequency - Inverse Document Frequency). Convierte texto en números, pero pondera la importancia de cada palabra

| Aspecto | CountVectorizer | TF-IDF |
|---------|-----------------|--------|
| **Valores** | Enteros (0, 1, 2, 3...) | Decimales (0.0 - 1.0) |
| **Enfoque** | "¿Cuántas veces aparece?" | "¿Qué tan importante es?" |
| **Palabras comunes** | Peso alto si aparecen mucho | Peso bajo automáticamente |
| **Palabras raras** | Peso según frecuencia | Peso alto si son distintivas |
| **Interpretación** | Muy fácil | Requiere entendimiento |
|Analogía Simple|¿Cuántas veces vio 'armadura'? → 5 veces| ¿Cuántas veces vio 'armadura' Y qué tan importante es esa palabra? → 'Armadura' aparece 5 veces, y es muy rara en otros textos, así que vale 0.85|



In [25]:
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#from sklearn.naive_bayes import MultinomialNB, ComplementNB
#from sklearn.metrics import f1_score
#==========================================
# Parámetros del Vectorizador (CountVectotizer)
countvect = CountVectorizer()
X_train_countvect = countvect.fit_transform(df_train['texto'])
X_test_countvect = countvect.transform(df_test['texto'])  # Solo transform, no fit

# Etiquetas
y_train_countvect = df_train['etiqueta']
y_test_countvect = df_test['etiqueta']

""" 
agrego el código de para tener una secuencia en el bloque 
en bloques previos calculamos los modelos utilizando el Vectorizador  TfidfVectorizer (TF-IDF)
"""
##==========================================b
## Parámetros del Vectorizador (TfidfVectorizer)
#tfidfvect = TfidfVectorizer()
#X_train = tfidfvect.fit_transform(df_train['texto'])
#X_test = tfidfvect.transform(df_test['texto'])  # Solo transform, no fit

## Etiquetas
#y_train = df_train['etiqueta']
#y_test = df_test['etiqueta']


# ******************************************************************************************
# modelos MultinomialNB y ComplementNB con vectorización CountVectorizer
#==============================
#  Multinomial Naive Bayes
# =============================

# Instanciar y entrenar modelo
#mnb_countvect = MultinomialNB(alpha=var_alpha)
mnb_countvect = MultinomialNB()
mnb_countvect.fit(X_train_countvect, y_train_countvect)
# Predicciones
y_pred_mnb_countvect = mnb_countvect.predict(X_test_countvect)
# F1-score macro
f1_mnb_countvect = f1_score(y_test_countvect, y_pred_mnb_countvect, average='macro')


# ==========================================
# Complement Naive Bayes
# ==========================================

# Instanciar y entrenar modelo
#cnb_countvect = ComplementNB(alpha=var_alpha)
cnb_countvect = ComplementNB()
#cnb_countvect = ComplementNB(norm=True) # Con normalización
cnb_countvect.fit(X_train_countvect, y_train_countvect)
# Predicciones
y_pred_cnb_countvect = cnb_countvect.predict(X_test_countvect)
# F1-score macro
f1_cnb_countvect = f1_score(y_test_countvect, y_pred_cnb_countvect, average='macro')


# ******************************************************************************************


print(f"   Predicciones realizadas MultinomialNB: {len(y_pred_cnb_countvect)} = {len(y_pred_cnb)}")
print(f"   Predicciones realizadas ComplementNB: {len(y_pred_cnb_countvect)} = {len(y_pred_cnb)} ")

# ==================================================
# COMPARACIÓN DE MODELOS CON VECTORIZACIÓN  CountVectorizer
# ==================================================
print("=" * 60)
print(f"COMPARACIÓN DE MODELOS CON VECTORIZACIÓN  CountVectorizer ")
print(f"MultinomialNB CountVectorizer   F1-score (macro): {f1_mnb_countvect}")
print(f"ComplementNB CountVectorizer     F1-score (macro): {f1_cnb_countvect}")


if f1_mnb_countvect > f1_cnb_countvect:
    print("Ganador: MultinomialNB")
elif f1_cnb_countvect > f1_mnb_countvect:
    print("Ganador: ComplementNB") 
else:
    print("** Empate **")
print("-" * 40)

# ================================================
# COMPARACIÓN DE MODELOS CON VECTORIZACIÓN TfidfVectorizer
# ================================================

print("COMPARACIÓN DE MODELOS CON VECTORIZACIÓN TfidfVectorizer")
print(f"MultinomialNB  TfidfVectorizer  F1-score (macro): {f1_mnb}")
print(f"ComplementNB   TfidfVectorizer  F1-score (macro): {f1_cnb}")

# *************************************
if f1_mnb > f1_cnb:
    print("Ganador: MultinomialNB")
elif f1_cnb > f1_mnb:
    print("Ganador: ComplementNB") 
else:
    print("** Empate **")
print("=" * 60)

   Predicciones realizadas MultinomialNB: 786 = 786
   Predicciones realizadas ComplementNB: 786 = 786 
COMPARACIÓN DE MODELOS CON VECTORIZACIÓN  CountVectorizer 
MultinomialNB CountVectorizer   F1-score (macro): 1.0
ComplementNB CountVectorizer     F1-score (macro): 1.0
** Empate **
----------------------------------------
COMPARACIÓN DE MODELOS CON VECTORIZACIÓN TfidfVectorizer
MultinomialNB  TfidfVectorizer  F1-score (macro): 0.5577733084525537
ComplementNB   TfidfVectorizer  F1-score (macro): 1.0
Ganador: ComplementNB


En este ejemplo podemos ver que CountVectorizer funcionó mejor con MultinomialNB a causa de : 
- El  dataset tiene vocabulario muy específico por clase (EL contexto literario de cada clase es muy distinto) esto dificulta que TF-IDF realice comparación en el vocabulario entre las clases. 
-  Adicionalmente se el modelo  MultinomialNB esta optimizado para ser utilizado con CountVectorizer. 

<h3><a href="#3">Punto 3</a></h3>


<div style="text-align: right;">
	<h1 style="display: inline-block;margin: 0;padding: 8px 16px;color: white;background: linear-gradient(to right,rgb(17, 75, 141), #4CAF50);border-radius: 12px;font-size: 1.8rem;box-shadow: 0 4px 10px rgba(0, 0, 0, 0.3);">Teoria</h1>
</div>





### Transponer la matriz documento-término

#### ¿Qué es la matriz documento-término?

Cuando usas un vectorizador como `CountVectorizer` o `TfidfVectorizer` sobre tu colección de documentos (corpus), el resultado es una **matriz documento-término**.

- **Filas:** Representan cada uno de tus documentos.
    
- **Columnas:** Representan cada término (palabra o n-grama) único que se encontró en tu vocabulario.
    
- **Valores:** Son los recuentos (`CountVectorizer`) o los valores TF-IDF (`TfidfVectorizer`) de cada término en cada documento.
    

Piensa en ella como una tabla donde cada fila es un documento y cada columna es una palabra, y las celdas indican la presencia/importancia de esa palabra en ese documento.

#### ¿Qué es transponerla?

**Transponer una matriz** significa intercambiar sus filas por sus columnas. Si tu matriz original tiene M filas (documentos) y N columnas (términos), la matriz transpuesta tendrá N filas (términos) y M columnas (documentos).

- **Matriz original (Documento-Término):**
    
    ```
            Palabra1  Palabra2  Palabra3 ...
    Doc1       0.5       0.2       0.0
    Doc2       0.1       0.7       0.3
    Doc3       0.0       0.1       0.9   ...
    ```
    
- **Matriz Transpuesta (Término-Documento):**
    
    ```
               Doc1      Doc2      Doc3 ...
    Palabra1   0.5       0.1       0.0
    Palabra2   0.2       0.7       0.1
    Palabra3   0.0       0.3       0.9  ...
    ```
    

#### ¿Cómo se obtiene una "vectorización de palabras"?

En la matriz transpuesta, cada **fila** ahora representa una palabra, y los valores en esa fila (`[0.5, 0.1, 0.0]` para `Palabra1` en el ejemplo) son su "vector". Este vector describe la **distribución de esa palabra a través de todos los documentos**. Palabras que tienen vectores similares (es decir, que aparecen en los mismos documentos con patrones de frecuencia/TF-IDF similares) se consideran semánticamente relacionadas o contextualmente parecidas.

#### ¿Cómo hacerlo en Python?

Después de haber vectorizado tu corpus (usando `CountVectorizer` o `TfidfVectorizer`), obtendrás una matriz dispersa (`X`). Para transponerla:



```Python
from sklearn.feature_extraction.text import TfidfVectorizer
# Suponiendo que 'corpus' es tu lista de documentos de texto
# ... (código para cargar y preparar df['texto'] como corpus) ...

# 1. Vectorizar el texto (ejemplo con TfidfVectorizer)
vectorizer = TfidfVectorizer(max_features=5000, stop_words='spanish') # Ajusta tus parámetros
X = vectorizer.fit_transform(corpus)

# 2. Obtener los nombres de las características (las palabras/n-gramas del vocabulario)
feature_names = vectorizer.get_feature_names_out()

# 3. Transponer la matriz
# Si X es una matriz dispersa (csr_matrix o csc_matrix), puedes usar .T
X_transposed = X.T

print(f"Forma original de la matriz (Documento-Término): {X.shape}")
print(f"Forma de la matriz transpuesta (Término-Documento): {X_transposed.shape}")
```


In [26]:
# Búsqueda de palabras mas comunes por cantidad de caracteres. 
import numpy as np
# min_length = cantidad de palabras (min_length=5
def analizar_palabras_por_longitud(top_n,vectorizer, X_matrix, min_length, df_datos=None, mostrar_documentos=True):
    """
    Analiza palabras filtradas por longitud mínima para evitar artículos/preposiciones
    
    Args:
        vectorizer: vectorizador entrenado
        X_matrix: matriz vectorizada
        min_length: longitud mínima de palabras
        df_datos: DataFrame con los datos (opcional, para mostrar documentos)
        mostrar_documentos: si mostrar en qué documentos aparece cada palabra
    """

    
    feature_names = vectorizer.get_feature_names_out()
    
    if hasattr(X_matrix, 'toarray'):
        frecuencias_totales = np.array(X_matrix.sum(axis=0)).flatten()
        X_dense = X_matrix.toarray()
    else:
        frecuencias_totales = X_matrix.sum(axis=0)
        X_dense = X_matrix
    
    # Filtrar palabras por longitud
    palabras_largas = []
    for palabra, freq in zip(feature_names, frecuencias_totales):
        if len(palabra) >= min_length:
            palabras_largas.append((palabra, freq))
    
    # Ordenar y mostrar top 15
    palabras_largas = sorted(palabras_largas, key=lambda x: x[1], reverse=True)
    
    print(f"TOP {top_n} palabras significativas (≥{min_length} caracteres):")
    
    for i, (palabra, frecuencia) in enumerate(palabras_largas[:top_n], 1):
        print(f"{i:2d}. '{palabra}' → {frecuencia:.2f}")
        
        # Mostrar documentos donde aparece la palabra si se solicita
        if mostrar_documentos and df_datos is not None:
            # Obtener índice de la palabra en el vocabulario
            palabra_idx = vectorizer.vocabulary_[palabra]
            
            # Encontrar documentos donde la palabra tiene valor > 0
            docs_con_palabra = []
            for doc_idx in range(X_dense.shape[0]):
                if X_dense[doc_idx, palabra_idx] > 0:
                    etiqueta = df_datos['etiqueta'].iloc[doc_idx]
                    valor = X_dense[doc_idx, palabra_idx]
                    docs_con_palabra.append((doc_idx, etiqueta, valor))
            
            # Ordenar por valor (frecuencia en el documento)
            docs_con_palabra = sorted(docs_con_palabra, key=lambda x: x[2], reverse=True)
            
            print(f"      Aparece en {len(docs_con_palabra)} documentos:")
            for doc_idx, etiqueta, valor in docs_con_palabra[:5]:  # Mostrar top 5
                print(f"        Etiqueta/Clase: {etiqueta} -> documento: {doc_idx} -> valor: {valor:.4f}")
            
            if len(docs_con_palabra) > 5:
                print(f"        ... y {len(docs_con_palabra) - 5} documentos más")
            print()
    
    return palabras_largas


#analisis=  analizar_palabras_por_longitud(tfidfvect, X_train)
#analisis = analizar_palabras_por_longitud(tfidfvect, X_train, df_datos=df, mostrar_documentos=True)
# 7 representa el numero de caracteres
analisis = analizar_palabras_por_longitud(15, tfidfvect, X_train,7, df_datos=df)
#analisis = analizar_palabras_por_longitud(15, countvect, X_train,7, df_datos=df)



TOP 15 palabras significativas (≥7 caracteres):
 1. 'quijote' → 19.64
      Aparece en 98 documentos:
        Etiqueta/Clase: don_quijote -> documento: 439 -> valor: 0.3433
        Etiqueta/Clase: don_quijote -> documento: 411 -> valor: 0.3232
        Etiqueta/Clase: don_quijote -> documento: 397 -> valor: 0.3197
        Etiqueta/Clase: don_quijote -> documento: 457 -> valor: 0.3125
        Etiqueta/Clase: don_quijote -> documento: 455 -> valor: 0.3105
        ... y 93 documentos más

 2. 'mientras' → 11.58
      Aparece en 446 documentos:
        Etiqueta/Clase: neuromante -> documento: 741 -> valor: 0.0917
        Etiqueta/Clase: neuromante -> documento: 740 -> valor: 0.0889
        Etiqueta/Clase: neuromante -> documento: 785 -> valor: 0.0829
        Etiqueta/Clase: neuromante -> documento: 757 -> valor: 0.0761
        Etiqueta/Clase: neuromante -> documento: 756 -> valor: 0.0761
        ... y 441 documentos más

 3. 'strybjorn' → 11.57
      Aparece en 168 documentos:
        Etiqu

In [27]:
# Búsqueda de palabras mas comunes entre todo los documentos 
def obtener_top_palabras_frecuentes(vectorizer, X_matrix, top_n=1):
    """
    Obtiene las palabras más frecuentes del dataset vectorizado
    
    Args:
        vectorizer: vectorizador ya entrenado (CountVectorizer o TfidfVectorizer)
        X_matrix: matriz vectorizada (documento-término)
        top_n: número de palabras a mostrar
    
    Returns:
        lista de tuplas (palabra, frecuencia_total)
    """
        
    # Obtener nombres de las características (palabras)
    feature_names = vectorizer.get_feature_names_out()
    
    # Sumar frecuencias por columna (cada columna es una palabra)
    if hasattr(X_matrix, 'toarray'):
        # Para matrices dispersas (sparse)
        frecuencias_totales = np.array(X_matrix.sum(axis=0)).flatten()
    else:
        # Para matrices densas
        frecuencias_totales = X_matrix.sum(axis=0)
    
    # Crear pares (palabra, frecuencia)
    palabra_frecuencia = list(zip(feature_names, frecuencias_totales))
    
    # Ordenar por frecuencia (mayor a menor)
    palabra_frecuencia_sorted = sorted(palabra_frecuencia, key=lambda x: x[1], reverse=True)
    
    # Mostrar TOP N
    print(f"TOP {top_n} PALABRAS MÁS FRECUENTES EN TODO EL TEXTO :")
    print("-" * 40)
    
    top_palabras = palabra_frecuencia_sorted[:top_n]
    
    for i, (palabra, frecuencia) in enumerate(top_palabras, 1):
        print(f"{i:2d}. '{palabra}' → {frecuencia:.2f} veces")
    
    #print()
    #print(f" Estadísticas del vocabulario:")
    #print(f"   Total de palabras únicas: {len(feature_names)}")
    #print(f"   Frecuencia máxima: {max(frecuencias_totales):.2f}")
    #print(f"   Frecuencia mínima: {min(frecuencias_totales):.2f}")
    #print(f"   Frecuencia promedio: {np.mean(frecuencias_totales):.2f}")
    
    return top_palabras


top_10 = obtener_top_palabras_frecuentes(tfidfvect, X_train, top_n=5)

TOP 5 PALABRAS MÁS FRECUENTES EN TODO EL TEXTO :
----------------------------------------
 1. 'de' → 250.16 veces
 2. 'que' → 152.95 veces
 3. 'la' → 149.30 veces
 4. 'el' → 110.17 veces
 5. 'en' → 89.58 veces


In [28]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def analizar_similaridad_palabra(X_transpuesta, vectorizer, palabra_objetivo, df_datos=None):
    """
    Analiza la similaridad de una palabra específica con todas las demás
    
    Args:
        X_transpuesta: matriz transpuesta (palabras x documentos)
        vectorizer: vectorizador entrenado
        palabra_objetivo: palabra a analizar
        df_datos: DataFrame opcional para contexto adicional
    
    Returns:
        diccionario con resultados del análisis
    """
    
    # Verificar si la palabra está en el vocabulario
    if palabra_objetivo not in vectorizer.vocabulary_:
        print(f"ERRO: La palabra '{palabra_objetivo}' no está en el vocabulario")
        return None
    
    print(f" análisis de similaridad de : --- {palabra_objetivo.upper()} ---")
    print("=" * 60)
    
    # Obtener índice de la palabra objetivo
    idx_palabra = vectorizer.vocabulary_[palabra_objetivo]
    ###feature_names = vectorizer.get_feature_names_out()
    
    print(f"Índice en vocabulario: {idx_palabra}")
    print(f"Contexto: Vector de {X_transpuesta.shape[1]} documentos")
    

    #--------------------------------
    #TF-IDF Importancia relativa de esa palabra en ese documento vector_palabra
    #TF: Qué tan frecuente es la palabra en ESE documento
    #IDF: Qué tan rara es la palabra en TODO el corpus
    #---------------------------------
    # Mostrar en qué documentos aparece la palabra objetivo
    if df_datos is not None:
        vector_palabra = X_transpuesta[idx_palabra].toarray().flatten() if hasattr(X_transpuesta, 'toarray') else X_transpuesta[idx_palabra]
        docs_con_palabra = []

        for doc_idx, valor in enumerate(vector_palabra):
            if valor > 0:
                etiqueta = df_datos['etiqueta'].iloc[doc_idx]
                docs_con_palabra.append((doc_idx, etiqueta, valor))
        
        docs_con_palabra = sorted(docs_con_palabra, key=lambda x: x[2], reverse=True)
        
        print(f"Aparece en {len(docs_con_palabra)} documentos:")
        for doc_idx, etiqueta, valor in docs_con_palabra[:5]:
            print(f"    Documento {doc_idx} ({etiqueta}) → valor  TF-IDF: {valor:.6f}")
            # 5 cantidad de resultados
        if len(docs_con_palabra) > 5:
            print(f"   ... y {len(docs_con_palabra) - 5} documentos más")
    
    print()
    
    # Calcular similaridad coseno con todas las palabras
    vector_objetivo = X_transpuesta[idx_palabra:idx_palabra+1]  # Mantener forma 2D
    similaridades = cosine_similarity(vector_objetivo, X_transpuesta)[0]
    
    # Estadísticas de similaridad
    print(f"ESTADÍSTICAS DE SIMILARIDAD DE LA PALABRA EN EL ESPACIO DE VECTORES:")
    print(f"   Similaridad de la palabra consigo misma: {similaridades[idx_palabra]}")
    print(f"   Similaridad de la palabra promedio: {np.mean(similaridades)}")
    print(f"   Similaridad de la palabra máxima (sin incluirse): {np.max(similaridades[similaridades < 1.0])}")
    print(f"   Similaridad de la palabra mínima: {np.min(similaridades)}")
    print()
  
    return


def analisis_completo_similaridad_palabras(X_train, vectorizer, palabras_buscar, df_datos=None):
    """
    Trasponemos la matriz, Verificamos que la palabra exista, realizamos la vectorización 
    """
    print(" ANÁLISIS DE SIMILARIDAD ENTRE PALABRAS ")
    print("=" * 70)
    print()
    
    # Transponer matriz
    X_transpuesta = X_train.T
    #  Matriz transpuesta: término-documento (filas=palabras, columnas=documentos)  
    print(f" Matriz original: documento-término {X_train.shape}")
    # Matriz transpuesta: término-documento (filas=palabras, columnas=documentos) 
    print(f" Matriz transpuesta: término-documento {X_transpuesta.shape}")
    print()
    
    # Verificar que todas las palabras están en el vocabulario
    print("VERIFICACIÓN DE PALABRAS EN VOCABULARIO:")
    print("-" * 50)
    
    palabras_encontradas = []
    for palabra in palabras_buscar:
        if palabra in vectorizer.vocabulary_:
            print(f" '{palabra}' - ENCONTRADA (índice: {vectorizer.vocabulary_[palabra]})")
            palabras_encontradas.append(palabra)
        else:
            print(f"ERRO: '{palabra}' - NO ENCONTRADA")
    
    if not palabras_encontradas:
        print("ERRO: No se encontraron palabras válidas para analizar")
        return None
    
    print(f"\n similaridad de  {len(palabras_encontradas)} palabras válidas")

    
    # Analizar cada palabra
    resultados = []
    for i, palabra in enumerate(palabras_encontradas):
        resultado = analizar_similaridad_palabra(X_transpuesta, vectorizer, palabra, df_datos)
        if resultado:
            resultados.append(resultado)
        
        if i < len(palabras_encontradas) - 1:  # Separador entre análisis
            print("\n" + "+" * 70 + "\n")
    
    return





palabras_buscar = ['momento', 'respondió', 'preguntó', 'después', 'también']
resultados = analisis_completo_similaridad_palabras(X_train, tfidfvect, palabras_buscar, df_datos=df)






 ANÁLISIS DE SIMILARIDAD ENTRE PALABRAS 

 Matriz original: documento-término (786, 20713)
 Matriz transpuesta: término-documento (20713, 786)

VERIFICACIÓN DE PALABRAS EN VOCABULARIO:
--------------------------------------------------
 'momento' - ENCONTRADA (índice: 13095)
 'respondió' - ENCONTRADA (índice: 17036)
 'preguntó' - ENCONTRADA (índice: 15425)
 'después' - ENCONTRADA (índice: 6351)
 'también' - ENCONTRADA (índice: 18868)

 similaridad de  5 palabras válidas
 análisis de similaridad de : --- MOMENTO ---
Índice en vocabulario: 13095
Contexto: Vector de 786 documentos
Aparece en 335 documentos:
    Documento 211 (lobo_espacial) → valor  TF-IDF: 0.094685
    Documento 210 (lobo_espacial) → valor  TF-IDF: 0.093013
    Documento 296 (lobo_espacial) → valor  TF-IDF: 0.085638
    Documento 291 (lobo_espacial) → valor  TF-IDF: 0.085069
    Documento 71 (lobo_espacial) → valor  TF-IDF: 0.073857
   ... y 330 documentos más

ESTADÍSTICAS DE SIMILARIDAD DE LA PALABRA EN EL ESPACIO DE V

In [29]:
# analizamos el similaridad de una palabra contra todos los documentos 
resultado = analizar_similaridad_palabra(X_train.T, tfidfvect, 'armadura', df)

 análisis de similaridad de : --- ARMADURA ---
Índice en vocabulario: 1758
Contexto: Vector de 786 documentos
Aparece en 125 documentos:
    Documento 336 (lobo_espacial) → valor  TF-IDF: 0.133060
    Documento 297 (lobo_espacial) → valor  TF-IDF: 0.129418
    Documento 337 (lobo_espacial) → valor  TF-IDF: 0.109153
    Documento 172 (lobo_espacial) → valor  TF-IDF: 0.108833
    Documento 281 (lobo_espacial) → valor  TF-IDF: 0.103617
   ... y 120 documentos más

ESTADÍSTICAS DE SIMILARIDAD DE LA PALABRA EN EL ESPACIO DE VECTORES:
   Similaridad de la palabra consigo misma: 0.9999999999999997
   Similaridad de la palabra promedio: 0.02932272246977407
   Similaridad de la palabra máxima (sin incluirse): 0.9999999999999997
   Similaridad de la palabra mínima: 0.0



A continuación buscamos una palabra específica y muestra en qué documentos aparece con mayor importancia (valor TF-IDF más alto)

In [30]:
# función con Matriz transpuesta, TfidfVectorizer entrenado, palabra y DataFrame
def explorar_contexto_palabra(X_transpuesta, vectorizer, palabra, df_datos, top_documentos=3):
    """
    Función auxiliar para explorar en detalle el contexto de una palabra,
    qué tipo de textos es más importante esta palabra
    """
    # verificamos si existe la palabra
    if palabra not in vectorizer.vocabulary_:
        print(f"Palabra '{palabra}' no encontrada")
        return
    # obtenemos información de las palabras
    # obtiene la posición de la palabra en el vocabulario
    idx_palabra = vectorizer.vocabulary_[palabra]
    #  extrae la fila correspondiente a esa palabra de la matriz transpuesta
    vector_palabra = X_transpuesta[idx_palabra].toarray().flatten() if hasattr(X_transpuesta, 'toarray') else X_transpuesta[idx_palabra]
    
    print(f"\nPalabra a buscar: '{palabra}'")
    print("-" * 40)
    
    # Documentos donde aparece con mayor peso
    docs_con_valor = []
    for doc_idx, valor in enumerate(vector_palabra):
        if valor > 0:
            etiqueta = df_datos['etiqueta'].iloc[doc_idx]
            texto_preview = df_datos['texto'].iloc[doc_idx][:100] + "..."
            docs_con_valor.append((doc_idx, etiqueta, valor, texto_preview))
    
    #ordenar por importancia
    docs_con_valor = sorted(docs_con_valor, key=lambda x: x[2], reverse=True)
    
    #mostramos resultados
    print(f"Top {top_documentos} documentos con mayor peso:")
    for i, (doc_idx, etiqueta, valor, texto) in enumerate(docs_con_valor[:top_documentos], 1):
        print(f"{i}. Doc {doc_idx} ({etiqueta}) - Peso: {valor:.4f}")
        print(f"   Texto: {texto}")
        print()


explorar_contexto_palabra(X_train.T, tfidfvect, 'respondió', df)


Palabra a buscar: 'respondió'
----------------------------------------
Top 3 documentos con mayor peso:
1. Doc 110 (lobo_espacial) - Peso: 0.0856
   Texto: estaba fuera de su lugar. Su cabello rojizo estaba cortado con un estilo que no era habitual en un i...

2. Doc 111 (lobo_espacial) - Peso: 0.0832
   Texto: de cerca. Sus rasgos eran finos y tenía un aspecto delicado e inteligente, más como el de un eskaldo...

3. Doc 633 (neuromante) - Peso: 0.0769
   Texto: piscina que caía en las baldosas. —Cath —dijo. —Lupus —respondió él después de una pausa. —¿Qué clas...

